## Cross-platform benchmark: BMF/GraphProt/iDeepE/DeepCLIP

In [1]:
import os 
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score, average_precision_score, precision_recall_curve, auc
import pickle
import seaborn as sns
from scipy.stats import wilcoxon

#### helper functions

In [2]:
def read_predictions(pred_file):
    pred = np.loadtxt(pred_file)
    return pred

In [3]:
def read_predictions_graphprot(pred_file):
    pred = pd.read_csv(pred_file, sep='\t', header=None)
    return pred.iloc[:,2].values

In [4]:
def read_sequence_ids_clip(file_path):
    bed_table = pd.read_csv(file_path, sep='\t', header=None)
    return bed_table.iloc[:,4].values

In [5]:
def compile_array_by_id(scores, ids):
    '''
    gets the scores and ids for positive sequences and combines it to 
    a single score array where each clip peak gets one score
    '''
    #make a dataframe and split it based on ids
    df = pd.DataFrame({'scores':scores, 'ids':ids})    
    final_scores = []
    for identifier, df in df.groupby('ids'):
        #keep the max value in each split
        final_score = df.loc[:,'scores'].mean()
        final_scores.append(final_score)
    return np.array(final_scores)

In [6]:
def get_new_scores_all(all_scores, clip_factor):
    bg_to_ps_ratio=1
    clip_all_file = os.path.join('../../cbscratch/eclip_encode/bed_splitted/',f'{clip_factor}_split_all.bed')
    ids = read_sequence_ids_clip(clip_all_file)

    
    final_scores = compile_array_by_id(all_scores, ids)
    
    no_seqs_pos = int(len(final_scores)/(bg_to_ps_ratio+1))    
    y_true = np.append(np.ones(no_seqs_pos), np.zeros(no_seqs_pos*bg_to_ps_ratio))

    return y_true, final_scores

In [7]:
def create_id_list(ids, bg_to_ps_ratio=10):
    id_list = []
    id_list.append(ids)
    new_ids = ids
    for i in range(bg_to_ps_ratio):
        new_ids = ids + new_ids[-1] + 1
        id_list.append(new_ids)
        
    return np.concatenate(id_list)

In [13]:
def plot_roc(fpr, tpr, labels, roc_auc, factor_name):
    lw = 2
    for i in range(len(fpr)):
        plt.plot(fpr[i], tpr[i], color=colors[i], lw=lw, label=f'{labels[i]} ROC curve (area = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{factor_name}\nReceiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(roc_plot_dir,f'{factor_name}.pdf'))
    plt.close()

### Train on HT-SELEX test on eCLIP

In [9]:
#directories where prediction files for each method is stored
graphprot_path = 'graphprot0/selex_eclip/'
ideepe_path = 'ideepe0/selex_eclip/'
deepclip_path = 'deepclip0/selex_eclip/'
bipartite_cs3_path = 'scripts/param/selex0/benchmark_selex_eclip_cs3/'
bipartite_cs4_path = 'scripts/param/selex0/benchmark_selex_eclip_cs4/'
bipartite_cs5_path = 'scripts/param/selex0/benchmark_selex_eclip_cs5/'

In [10]:
all_combos_table = pd.read_csv('../../cbscratch/mapping_selex_eclip_allcombinations.txt', sep='\t', header=None)
selex_factors = all_combos_table.iloc[:,0].values
eclip_factors = all_combos_table.iloc[:,1].values
combos = [f'{a}_{b}' for a,b in zip(selex_factors, eclip_factors)]

In [12]:
datasets = ['BMF cs3','BMF cs4','BMF cs5','GraphProt','iDeepE', 'DeepCLIP']
colors = ['#6baed6','#3182bd','#08519c','#fb9a99','#b2df8a','#fdbf6f']
no_experiments = len(datasets)

In [14]:
auc_list = []
ap_list = []
roc_plot_dir = 'plots/eclip/roc_plots/'

for i, combo in enumerate(combos):
    clip_factor = eclip_factors[i]
    
    fpr = []
    tpr = []
    labels = []
    factor_auc = []

    bipartite_cs3_file = os.path.join(bipartite_cs3_path, f'{combo}.predictions')
    bipartite_cs4_file = os.path.join(bipartite_cs4_path, f'{combo}.predictions')
    bipartite_cs5_file = os.path.join(bipartite_cs5_path, f'{combo}.predictions')
    graphprot_file = os.path.join(graphprot_path, f'{combo}_eclip.predictions')
    ideepe_file = os.path.join(ideepe_path, f'{combo}_predictions.txt')
    deepclip_file = os.path.join(deepclip_path, f'{combo}_predictions.tsv')

    bipartite_cs3_exists = os.path.isfile(bipartite_cs3_file) 
    bipartite_cs4_exists = os.path.isfile(bipartite_cs4_file) 
    bipartite_cs5_exists = os.path.isfile(bipartite_cs5_file) 
    graphprot_exists = os.path.isfile(graphprot_file) 
    ideepe_exists = os.path.isfile(ideepe_file) 
    deepclip_exists = os.path.isfile(deepclip_file) 
    

    if bipartite_cs3_exists:
        y_bipartite3 = read_predictions(bipartite_cs3_file)
        y_true, y_bipartite3 = get_new_scores_all(y_bipartite3, clip_factor)
        
        auc_bipartite3 = roc_auc_score(y_true, y_bipartite3)
        
        #plotting
        fpr0, tpr0, _ = roc_curve(y_true, y_bipartite3)
        fpr.append(fpr0)
        tpr.append(tpr0)
        labels.append('bipartite_cs3')
        factor_auc.append(auc_bipartite3)
        
        #print(y_bipartite.shape, fpr0.shape)
        
        ap_bipartite3 = average_precision_score(y_true, y_bipartite3)
        
        precision, recall, thresholds = precision_recall_curve(y_true, y_bipartite3)
    else:
        print(f'bipartite predictions for {combo} not found (runtime error)')
        auc_bipartite3 = np.nan
        ap_bipartite3 = np.nan
    
    if bipartite_cs4_exists:
        y_bipartite4 = read_predictions(bipartite_cs4_file)
        y_true, y_bipartite4 = get_new_scores_all(y_bipartite4, clip_factor)
        
        auc_bipartite4 = roc_auc_score(y_true, y_bipartite4)
        
        #plotting
        fpr0, tpr0, _ = roc_curve(y_true, y_bipartite4)        
        fpr.append(fpr0)
        tpr.append(tpr0)
        labels.append('bipartite_cs4')
        factor_auc.append(auc_bipartite4)
        
        ap_bipartite4 = average_precision_score(y_true, y_bipartite4)
        precision, recall, thresholds = precision_recall_curve(y_true, y_bipartite4)
    else:
        print(f'bipartite cs4 predictions for {combo} not found (runtime error)')
        auc_bipartite4 = np.nan
        ap_bipartite4 = np.nan
    
    if bipartite_cs5_exists:
        y_bipartite5 = read_predictions(bipartite_cs5_file)
        y_true, y_bipartite5 = get_new_scores_all(y_bipartite5, clip_factor)
        
        auc_bipartite5 = roc_auc_score(y_true, y_bipartite5)
        
        #plotting
        fpr0, tpr0, _ = roc_curve(y_true, y_bipartite5)        
        fpr.append(fpr0)
        tpr.append(tpr0)
        labels.append('bipartite_cs5')
        factor_auc.append(auc_bipartite5)
        
        ap_bipartite5 = average_precision_score(y_true, y_bipartite5)
        precision, recall, thresholds = precision_recall_curve(y_true, y_bipartite5)
    else:
        print(f'bipartite cs5 predictions for {combo} not found (runtime error)')
        auc_bipartite5 = np.nan
        ap_bipartite5 = np.nan
                
    if graphprot_exists:
        y_graphprot = read_predictions_graphprot(graphprot_file)
        y_true, y_graphprot = get_new_scores_all(y_graphprot, clip_factor)
        
        #plotting
        auc_graphprot = roc_auc_score(y_true, y_graphprot)
        fpr0, tpr0, _ = roc_curve(y_true, y_graphprot)
        fpr.append(fpr0)
        tpr.append(tpr0)
        labels.append('graphprot')
        factor_auc.append(auc_graphprot)
        
        ap_graphprot = average_precision_score(y_true, y_graphprot)
        precision, recall, thresholds = precision_recall_curve(y_true, y_graphprot)
    else:
        print(f'graphprot predictions for {combo} not found (runtime error)')
        auc_graphprot = np.nan
        ap_graphprot = np.nan


    if ideepe_exists:
        y_ideepe = read_predictions(ideepe_file)
        y_true, y_ideepe = get_new_scores_all(y_ideepe, clip_factor)
        
        #plotting
        auc_ideepe = roc_auc_score(y_true, y_ideepe)
        fpr0, tpr0, _ = roc_curve(y_true, y_ideepe)
        fpr.append(fpr0)
        tpr.append(tpr0)
        labels.append('iDeepE')
        factor_auc.append(auc_ideepe)
        
        #print(y_ideepe.shape, fpr0.shape)
        ap_ideepe = average_precision_score(y_true, y_ideepe)
        precision, recall, thresholds = precision_recall_curve(y_true, y_ideepe)

    else:
        print(f'ideepe predictions for {combo} not found (runtime error)')
        auc_ideepe = np.nan
        ap_ideepe = np.nan

        
    if deepclip_exists:
        y_deepclip = read_predictions_graphprot(deepclip_file)
        y_true, y_deepclip = get_new_scores_all(y_deepclip, clip_factor)
        
        #plotting
        auc_deepclip = roc_auc_score(y_true, y_deepclip)
        fpr0, tpr0, _ = roc_curve(y_true, y_deepclip)
        fpr.append(fpr0)
        tpr.append(tpr0)
        labels.append('DeepCLIP')
        factor_auc.append(auc_deepclip)
        
        ap_deepclip = average_precision_score(y_true, y_deepclip)
        precision, recall, thresholds = precision_recall_curve(y_true, y_deepclip)
    else:
        print(f'deepclip predictions for {combo} not found (runtime error)')
        auc_deepclip = np.nan
        ap_deepclip = np.nan
    
    #print(combo)
    plot_roc(fpr, tpr, labels, factor_auc, combo)
    
    auc_list.append([auc_bipartite3, auc_bipartite4, auc_bipartite5, auc_graphprot, auc_ideepe, auc_deepclip])  
    ap_list.append([ap_bipartite3, ap_bipartite4, ap_bipartite5, ap_graphprot, ap_ideepe, ap_deepclip])

deepclip predictions for KHDRBS1-rep1_KHDRBS1_K562 not found (runtime error)
deepclip predictions for ZC3H8-rep0_ZC3H8_K562 not found (runtime error)
deepclip predictions for ZC3H8-rep1_ZC3H8_K562 not found (runtime error)


In [15]:
plotting_dir = 'plots/eclip'
pickle.dump([auc_list, ap_list], open(os.path.join(plotting_dir,'metrics.pkl'), 'wb' ))

#auc_list, ap_list = pickle.load(open(os.path.join(plotting_dir,'metrics.pkl'), 'rb' ))

### Split by factor

In [16]:
def combine_reps_plot(metric_list, metric_name):
    metric_df = pd.DataFrame(metric_list, columns=datasets)
    metric_df['factors'] = [c.split('-')[0] for c in combos]

    metric_by_factor = {}
    metric_by_factor_range = []
    for label, df in metric_df.groupby('factors'):
        values = df.replace(0, np.NaN).mean()
        metric_by_factor_range.append((df.replace(0, np.NaN).min(),df.replace(0, np.NaN).max()))
        metric_by_factor[label] = values

    metric_by_factor = pd.DataFrame(metric_by_factor).T.loc[:,datasets].values

    metric_by_factor_max = [np.argmax(arr) for arr in np.array(metric_by_factor)]
    _ = plt.hist(metric_by_factor_max, bins=np.arange(-0.5,3.5,1), rwidth=0.8)
    _ = plt.xticks(np.arange(0,3,1), datasets, rotation=90)
    plt.title(f'best at {metric_name} (by factor)')
    plt.savefig(os.path.join(plotting_dir,f'{metric_name}_by_factor_summary.pdf'), bbox_inches='tight')
    plt.close()
    
    fig, ax = plt.subplots(figsize=(4,10))
    
    for i in range(len(datasets)):
        y_pos = np.arange(len(metric_by_factor)) - 0.3 + 0.17*i
        auc_i = np.array(metric_by_factor)[:,i]
        ax.barh(y_pos, auc_i, align='center', height=0.15, label=datasets[i], color=colors[i], edgecolor='black')

    best_colors = [colors[i] for i in metric_by_factor_max]
    ax.scatter([0.95]*len(metric_by_factor), np.arange(len(metric_by_factor)), color=best_colors, s=50)
    ax.vlines(x=0.5, ymin=-1, ymax=len(metric_by_factor)+1, ls='--')

    ax.set_yticks(y_pos-0.3)
    ax.set_yticklabels(metric_df.loc[:,'factors'].unique())
    ax.invert_yaxis()  # labels read top-to-bottom

    ax.set_ylim(len(metric_by_factor)+1, -1)

    ax.legend(loc=(1.1,0.9))
    ax.set_title(f'{metric_name} (by factor)')
    
    pickle.dump([metric_df.loc[:,'factors'].unique(), metric_by_factor, metric_by_factor_range], open(os.path.join(plotting_dir,f'{metric_name}_byfactor.pkl'), 'wb' ))

    plt.savefig(os.path.join(plotting_dir,f'{metric_name}_byfactor_selex_eclip_all.pdf'), bbox_inches='tight')
    plt.close()

In [17]:
metric_list = ap_list
metric_name = 'AP'
combine_reps_plot(metric_list, metric_name)

metric_list = auc_list
metric_name = 'AUROC'
combine_reps_plot(metric_list, metric_name)